In [1]:
print "Importing packages"

import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn as learn
import xgboost
import pandas as pd
import numpy as np
import re


Importing packages


Very simple neural net model.

It seems like California, New York, Texas & Florida have the largest number of projects.
Grades PreK-2 had the most number of projects, & the number of projects decrease monotonically with increasing age of students. Grades 9-12 have the fewest number of projects. I must say that this came as a surprise, I would hae thought seniors would need more funding.
There seems to be the following project categories:-

Applied Learning
Health & Sports
History & Civics
Math & Science
Music & The Arts
Special Needs
Warmth, Care & Hunger

But the majority of projects are tagged with multiple categories, hence there are lot more different project categories in the data (combinations of the base categories).

Math & Science, Literacy & Language & Health & Sport seem to be leaders in terms of project count.

In [2]:
#reading data

train= pd.read_csv("/home/souravg/train.csv")

print train.shape
test=pd.read_csv("/home/souravg/test.csv") 
print test.shape

resource= pd.read_csv("/home/souravg/resources.csv")

print train.head(7)

print resource.head(7)


joined= pd.merge(train,resource,how='inner',on='id',sort=False,)

joined_test=pd.merge(test,resource,how='inner',on='id',sort=False,)


print joined.columns

#print joined.head(5)

#print joined[0:2]
#train.school_state.astype("category")

#train.project_grade_category.astype("category")

#train.project_subject_categories.astype("category")

#train.project_subject_subcategories.astype("category")




(182080, 16)


/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(78035, 15)
        id                        teacher_id teacher_prefix school_state  \
0  p036502  484aaf11257089a66cfedc9461c6bd0a            Ms.           NV   
1  p039565  df72a3ba8089423fa8a94be88060f6ed           Mrs.           GA   
2  p233823  a9b876a9252e08a55e3d894150f75ba3            Ms.           UT   
3  p185307  525fdbb6ec7f538a48beebaa0a51b24f            Mr.           NC   
4  p013780  a63b5547a7239eae4c1872670848e61a            Mr.           CA   
5  p063374  403c6783e9286e51ab318fba40f8d729           Mrs.           DE   
6  p103285  4e156c5fb3eea2531601c8736f3751a7           Mrs.           MO   

  project_submitted_datetime project_grade_category  \
0        2016-11-18 14:45:59          Grades PreK-2   
1        2017-04-26 15:57:28             Grades 3-5   
2        2017-01-01 22:57:44             Grades 3-5   
3        2016-08-12 15:42:11             Grades 3-5   
4        2016-08-06 09:09:11             Grades 6-8   
5        2016-11-05 10:01:51          Grades PreK

In [3]:
joined['Total_sum']= joined.quantity*joined.price
joined_test['Total_sum']= joined_test.quantity*joined_test.price
#print joined.head(3)

print '-------------------------------'

#print joined_test.head(3)


-------------------------------


In [4]:
sumdf= joined.groupby('id',as_index=False).agg({'Total_sum':'sum'})

sumdf_test=joined_test.groupby('id',as_index=False).agg({'Total_sum':'sum'})

#print sumdf.head(3)
print '-----------------------'
#print sumdf_test.head(3)

#print joined.Total_sum.head(3)

-----------------------


In [5]:
final_train= pd.merge(train,sumdf,on='id',how='inner',sort=False,)
final_test=pd.merge(test,sumdf_test,on='id',how='inner',sort=False,)

#print final_train.head(3)
print '-------------------'
#print final_test.head(3)

-------------------


In [6]:
train=final_train
test=final_test

ftest=test.drop([u'id', u'teacher_id', u'teacher_prefix',u'project_title', u'project_essay_1', u'project_essay_2',
       u'project_essay_3', u'project_essay_4', u'project_resource_summary','project_submitted_datetime','school_state'],1)

ftrain=train.drop([u'id', u'teacher_id', u'teacher_prefix',u'project_title', u'project_essay_1', u'project_essay_2',
       u'project_essay_3', u'project_essay_4', u'project_resource_summary','project_submitted_datetime','school_state'],1)

#print ftrain.head(4)

target=ftrain.project_is_approved

ftrain=ftrain.drop(['project_is_approved'],1)

#print ftest.head(4)

#print target.head(5)
copy=pd.DataFrame(pd.concat([ftrain,ftest]))
print copy.head(3)

#copy["school_state"]=copy["school_state"].astype('category')
copy["project_grade_category"]=copy["project_grade_category"].astype('category')
copy["project_subject_categories"]=copy["project_subject_categories"].astype('category')
copy['project_subject_subcategories']=copy['project_subject_subcategories'].astype('category')

#print 'copy',copy["school_state"].describe()
print 'copy',copy["project_grade_category"].describe()
print 'copy',copy["project_subject_categories"].describe()
print 'copy',copy['project_subject_subcategories'].describe()
print '[[[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]]]'

ftrain= copy[0:182080]
ftest= copy[182080:]


#ftest["school_state"]=ftest["school_state"].astype('category')
#ftest["project_grade_category"]=ftest["project_grade_category"].astype('category')
#ftest["project_subject_categories"]=ftest["project_subject_categories"].astype('category')
#ftest["project_subject_subcategories"]=ftest["project_subject_subcategories"].astype('category')

print '99999999999999999999999999999999999999999999999'
ftest=pd.get_dummies(ftest,prefix=['project_grade_category','project_subject_categories','project_subject_subcategories'])

print "test shape=",ftest.shape

#ftrain["school_state"]=ftrain["school_state"].astype('category')
#ftrain["project_grade_category"]=ftrain["project_grade_category"].astype('category')
#ftrain["project_subject_categories"]=ftrain["project_subject_categories"].astype('category')
#ftrain["project_subject_subcategories"]=ftrain["project_subject_subcategories"].astype('category')

print '99999999999999999999999999999999999999999999999'

ftrain=pd.get_dummies(ftrain,prefix=['project_grade_category','project_subject_categories','project_subject_subcategories'])

print "train shape=",ftrain.shape

#print '*************************************printing category mapping'




#l=list(zip(ftest.school_state.cat.codes, ftest.school_state.cat.categories))
#for i in l:
#    print i

#print '*****************Converting categoricals to dummies*****************'

#ftest=pd.get_dummies(ftest,prefix=['state','grade','categories','subcategories'])

print '***************************************'


  project_grade_category           project_subject_categories  \
0          Grades PreK-2                  Literacy & Language   
1             Grades 3-5    Music & The Arts, Health & Sports   
2             Grades 3-5  Math & Science, Literacy & Language   

            project_subject_subcategories  \
0                                Literacy   
1            Performing Arts, Team Sports   
2  Applied Sciences, Literature & Writing   

   teacher_number_of_previously_posted_projects  Total_sum  
0                                            26     899.94  
1                                             1     400.00  
2                                             5     469.99  
copy count            260115
unique                4
top       Grades PreK-2
freq             105361
Name: project_grade_category, dtype: object
copy count                  260115
unique                     51
top       Literacy & Language
freq                    56034
Name: project_subject_categories, dtype: obj

In [7]:



print '*****************************************'


print '========================================== Normalization of project count & total sum'

projectcount_mean=ftrain['teacher_number_of_previously_posted_projects'].mean()
projectcount_sd=ftrain['teacher_number_of_previously_posted_projects'].std()

totalsum_mean=ftrain['Total_sum'].mean()
totalsum_sd=ftrain['Total_sum'].std()

print projectcount_mean,' ',projectcount_sd

ftrain['Total_sum']=(ftrain['Total_sum']-totalsum_mean)/totalsum_sd
ftrain['teacher_number_of_previously_posted_projects']= (ftrain['teacher_number_of_previously_posted_projects']-projectcount_mean)/projectcount_sd

print totalsum_mean,' ',totalsum_sd

ftest['Total_sum']=(ftest['Total_sum']-totalsum_mean)/totalsum_sd
ftest['teacher_number_of_previously_posted_projects']= (ftest['teacher_number_of_previously_posted_projects']-projectcount_mean)/projectcount_sd


print '-------------------------------- Normalization done'


from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn import ensemble
from sklearn.neural_network import MLPClassifier

#abclf= ensemble.AdaBoostClassifier(n_estimators=1000,)
#rfclf= ensemble.RandomForestClassifier(n_estimators=1000,criterion='entropy',max_depth=5,warm_start=True,)
#gbclf= ensemble.GradientBoostingClassifier(n_estimators=1000,max_depth=4,min_samples_split=4,subsample=.3)

#clf= ensemble.RandomForestClassifier(n_estimators=500,criterion='entropy',max_depth=3,warm_start=True,)
#clf = AdaBoostClassifier(n_estimators=500, learning_rate= 0.4)
print 'Different hyper parameter combos have been tried. currently trying 2nd combo'
#1) clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(200, 100), random_state=1,max_iter=1000)-> score= 0.51
clf = MLPClassifier(solver='adam', activation='tanh',alpha=2e-4,hidden_layer_sizes=(100, 25), max_iter=500)
print "Hola"



clf.fit(ftrain,target)


#abclf.fit(ftrain, target)
#rfclf.fit(ftrain, target)
#gbclf.fit(ftrain, target)

#print 'Feature importance from adaboost='
#print abclf.feature_importances_

#print 'Feature importance from random forest='
#print rfclf.feature_importances_

#print 'Feature importance from gradient boosting='
#print gbclf.feature_importances_

#tabresult=abclf.predict_proba(ftrain)
#trfresult=rfclf.predict_proba(ftrain)
#tgbresult=gbclf.predict_proba(ftrain)


#abresult=abclf.predict_proba(ftest)
#rfresult=rfclf.predict_proba(ftest)
#gbresult=gbclf.predict_proba(ftest)

print "finito"

*****************************************
========================================== Normalization of project count & total sum
11.2370551406   28.0160860194
545.748957985   548.198712507
-------------------------------- Normalization done
Different hyper parameter combos have been tried. currently trying 2nd combo
Hola
finito


In [8]:

result=clf.predict_proba(ftest)

r_df=pd.DataFrame(dict(id=test.id,project_is_approved=result[:,1]))

print r_df.shape
print r_df.head(30)
#print rfclf.feature_importances_


r_df.to_csv('/home/souravg/output6.3.csv',index=False)


(78035, 2)
         id  project_is_approved
0   p233245             0.854758
1   p096795             0.862844
2   p236235             0.822860
3   p233680             0.887984
4   p171879             0.628737
5   p016071             0.905951
6   p099906             0.905413
7   p200236             0.933280
8   p129452             0.950987
9   p186652             0.862127
10  p257992             0.875311
11  p055177             0.857303
12  p069502             0.782505
13  p099455             0.868759
14  p226195             0.885843
15  p048432             0.933052
16  p028074             0.821227
17  p188220             0.847140
18  p213507             0.875556
19  p006068             0.904281
20  p124112             0.883944
21  p190450             0.860831
22  p180156             0.886172
23  p025868             0.846056
24  p134875             0.784495
25  p169259             0.862139
26  p124445             0.848276
27  p090153             0.936431
28  p086796             0.850403